In [ ]:
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('words')


from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tuning_model_class import w2v_tuning
from transformers import  ElectraTokenizerFast
from nltk.corpus import wordnet, stopwords, words
import json
import pandas as pd

In [ ]:
vocabulary = words.words()

In [ ]:
w2vmodel = w2v_tuning()
w2vmodel.set_vector_path("w2v.42B.300d.txt",'GloVe.wordvectors')
w2vmodel.CreateW2V_Model()

In [ ]:
w2vmodel.model.wv.most_similar('event',topn=5)

In [ ]:
# db_schema = ['zipcode','state','company','product','date']
db_schema = ['id','name','age','song','show','order']

In [ ]:
target_word = 'How many singer in ABC concert?'
# target_word = 'Who play LOVE in ABC tour?'
target_word = 'Which name of singer that play in event?'

In [ ]:
def preprocess_question(text:str):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [WordNetLemmatizer().lemmatize(word, wordnet.NOUN) for word in tokens if (word not in stop_words and word.isalnum())]
    return filtered_tokens

In [ ]:
def tokenize_column(col_list:list):
    tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-large-discriminator")
    token_col = [tokenizer.tokenize(col_name) for col_name in col_list]
    return token_col

In [ ]:
for word in preprocess_question(target_word):
    similarities = {}
    for w in db_schema:
        similarities[w] = w2vmodel.model.wv.similarity(word,w)

    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    print(f"'{word}' similar '{sorted_similarities[0][0]} {sorted_similarities[0][1]:.2f}'")

In [ ]:
# can can redirect to fullcolumn name (with key)

In [ ]:
db_id = "concert_singer"
questions = []

with open("spider/dev.json", 'r') as json_file:
    data = json.load(json_file)

for element in data:
    if element['db_id'] == db_id:
        questions.append(element['question'])

In [ ]:
import sqlite3

# Specify the full path to the SQLite database file
db_file_path = 'data/concert_singer/concert_singer.sqlite'
db_dict = {}
col_to_tables = {}

# Connect to the database
connection = sqlite3.connect(db_file_path)

# Create a cursor object
cursor = connection.cursor()

# Query the SQLite master table to get the list of table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
table_names = cursor.fetchall()

# Iterate through the table names
for table in table_names:
    table_name = table[0]
    print(f"Table: {table_name}")

    # Query the PRAGMA statement to get column information for the current table
    cursor.execute(f"PRAGMA table_info({table_name})")

    # Fetch column information
    column_info = cursor.fetchall()

    # Extract and print column names for the current table
    column_names = [col[1] for col in column_info]
    print("Columns:", column_names)
    # print("Tokenize columns:", tokenize_column(column_names))
    db_dict[table_name] = column_names
    
    for col in column_names:
        col_to_tables[col] = table_name

    print("\n")

# Close the cursor and the connection
cursor.close()
connection.close()


In [ ]:
db_dict

In [ ]:
col_to_tables

In [ ]:
questions[:10]

In [ ]:
for q in questions[:10]:
    q_tokens = preprocess_question(q)
    print(q_tokens)

In [135]:
def compare_word_col(df, question:str):
    
    q_tokens = preprocess_question(question)
    for qtoken in q_tokens:
        is_value_exist = df['Q_token'].isin([q_tokens])

        if not is_value_exist.any():    #word is not exist
            list_score = [qtoken]
            tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-large-discriminator")
            for col_name in list(df.columns):
                if col_name == "Q_token":
                    continue
                score = 0
                for ctoken in tokenizer.tokenize(col_name):
                    similar_score = w2vmodel.model.wv.similarity(ctoken,qtoken)
                    if score < similar_score:
                        score = similar_score
                
                list_score.append(score)
                

            df.loc[len(df.index)] = list_score
            # Reset the index to reindex the DataFrame
            df.reset_index(drop=True, inplace=True)


    return df

In [136]:
cols = ['Q_token'] + list(col_to_tables.keys())
df = pd.DataFrame(columns=cols)

compare_word_col(df,q)

,Q_token,Stadium_ID,Location,Name,Capacity,Highest,Lowest,Average,Singer_ID,Country,Song_Name,Song_release_year,Age,Is_male,concert_ID,concert_Name,Theme,Year
0,many,0.353882,0.421393,0.516184,0.409436,0.467571,0.381741,0.432902,0.353882,0.586557,0.516184,0.591357,0.506382,0.614467,0.378397,0.516184,0.416133,0.591357
1,singer,0.227184,0.229849,0.415831,0.173727,0.209600,0.120408,0.225402,1.000000,0.421359,0.633667,0.633667,0.350259,0.410767,0.545982,0.545982,0.273864,0.375702


In [ ]:
def map_column_table(db_dict,col_to_table,column_name):
    
    return

In [137]:
cols = ['Q_token'] + list(col_to_tables.keys())
df = pd.DataFrame(columns=cols)

q = questions[0]
print("Question:", q)

q_tokens = preprocess_question(q)
print("Question tokens:", q_tokens)



Question: How many singers do we have?
Question tokens: ['many', 'singer']
